In [48]:
# try the imports
import os
import torch
import math
import numpy as np
import pandas as pd
import sys
import random
from descarteslabs import Storage, Auth
sys.path.append("../")
import constants
from tqdm.notebook import tqdm
sys.path.append('../UI_Labeling')
import config
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

!{sys.executable} -m pip install opencv-contrib-python
import cv2

from PIL import Image
import shutil

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [49]:
paths=[
    #'bangladesh_dl_dataset',
    #'india_dl_dataset',
    'Upsampled_high_sampled_india_dl_dataset',
    'Upsampled_updatedModel_state_sampled_pakistan_dl_dataset',
    'Upsampled_updatedModel_state_sampled_nepal_dl_dataset'
    ]

In [50]:
prefix='v2_'

In [53]:
for path in tqdm(paths):
    if not os.path.exists('Image_Datasets/'+prefix+path):
        os.mkdir('Image_Datasets/'+prefix+path)
        os.mkdir('Image_Datasets/'+prefix+path+'/positives/')
        os.mkdir('Image_Datasets/'+prefix+path+'/negatives/')
    
    shutil.copy('Image_Datasets/'+path+'/metadata.csv','Image_Datasets/'+prefix+path+'/metadata.csv')
    
    for folder in ['/positives/','/negatives/']:
        for posfile in tqdm(os.listdir('Image_Datasets/'+path+folder)):
            if posfile == '.ipynb_checkpoints':
                continue;
            num,_=posfile.split('.')
            file_name='Image_Datasets/'+path+folder+posfile
            
            if path in ['bangladesh_dl_dataset','india_dl_dataset']:
                img = np.load(file_name)[1:4]
                sr=cv2.dnn_superres.DnnSuperResImpl_create()
                mod_path="ESPCN_x4.pb"
                sr.readModel(mod_path)
                sr.setModel('espcn',4)
                rgb=np.transpose(img)
                rgb=rgb-np.min(rgb)
                rgb=rgb/np.max(rgb)
                rgb = np.uint8(255 * rgb)
                result=sr.upsample(rgb)
                result=result[:,:,[2,1,0]]
                result=np.transpose(result,[2,0,1])
                #img = Image.fromarray(result, "RGB")
            else:
                img=np.transpose(np.load(file_name),[2,0,1])
                img=img[[0,2,1]]
                result=img
                #img = Image.fromarray(np.load(file_name)[:,:,[0,2,1]], "RGB")
            np.save('Image_Datasets/'+prefix+path+folder+posfile,result)
            #img.save('Image_Datasets/jpeg_'+path+folder+num+'.jpeg')

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/858 [00:00<?, ?it/s]

  0%|          | 0/4798 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/1739 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/327 [00:00<?, ?it/s]